In [1]:
import numpy as np


#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:

X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)



In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]


In [12]:
X_train_pairs = []
y_train_pair_lables = []
for Xtnn, ytnn, camId_t in zip(X_train, y_train, camId_train):
    p = 0
    for Xtnn2, ytnn2, camId_t2 in zip(X_train, y_train, camId_train):
        if ((camId_t == camId_t2) and (ytnn == ytnn2)):
            continue
        q = 0
        for Xtnn3, ytnn3, camId_t3 in zip(X_train, y_train, camId_train):
            if ((camId_t2 == camId_t3) and (ytnn2 == ytnn3)):
                continue
            else:
                X_train_pairs.append((Xtnn, Xtnn2, Xtnn3))
                if((ytnn == ytnn2) and (ytnn == ytnn3)):
                    y_train_pair_lables.append((0,0))
                elif(((ytnn == ytnn2) and (ytnn != ytnn3))):
                    y_train_pair_lables.append((0,1))
                elif(((ytnn != ytnn2) and (ytnn == ytnn3))):
                    y_train_pair_lables.append((1,0))
                else:
                    y_train_pair_lables.append((1,1))
            q = q + 1
            if q > 2:
                break
        p = p + 1
        if p > 2:
            break
y_train_pair_lables = np.array(y_train_pair_lables)
X_train_pairs = np.array(X_train_pairs)

MemoryError: 

In [13]:
y_train_pair_lables

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [1, 1],
       [1, 1],
       [1, 1]])

In [14]:
X_train_pairs

array([[[0.13791114, 1.12505555, 0.05404324, ..., 0.10747871,
         0.04081609, 0.68009406],
        [0.14378065, 0.75697702, 0.        , ..., 0.07771244,
         0.34392792, 0.91535425],
        [0.13791114, 1.12505555, 0.05404324, ..., 0.10747871,
         0.04081609, 0.68009406]],

       [[0.13791114, 1.12505555, 0.05404324, ..., 0.10747871,
         0.04081609, 0.68009406],
        [0.14378065, 0.75697702, 0.        , ..., 0.07771244,
         0.34392792, 0.91535425],
        [0.02507781, 0.9057585 , 0.00346441, ..., 0.16763815,
         0.14764351, 0.2700713 ]],

       [[0.13791114, 1.12505555, 0.05404324, ..., 0.10747871,
         0.04081609, 0.68009406],
        [0.14378065, 0.75697702, 0.        , ..., 0.07771244,
         0.34392792, 0.91535425],
        [0.09923808, 1.09122825, 0.01889733, ..., 0.07981343,
         0.04958951, 0.37923682]],

       ...,

       [[0.46024311, 2.31024432, 1.20531154, ..., 0.62753201,
         1.22394812, 0.        ],
        [0.09923808, 

In [15]:
len(X_train_pairs)

66312

In [17]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=200, alpha=1e-4,
                    solver='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

In [18]:
mlp.fit(X_train, y_train)

Iteration 1, loss = 7.78552994
Iteration 2, loss = 6.79945158
Iteration 3, loss = 6.30535584
Iteration 4, loss = 6.13245759
Iteration 5, loss = 6.00453691
Iteration 6, loss = 5.91138517
Iteration 7, loss = 5.85420269
Iteration 8, loss = 5.75260260
Iteration 9, loss = 5.58447545
Iteration 10, loss = 5.54572988
Iteration 11, loss = 5.52440419
Iteration 12, loss = 5.44266349
Iteration 13, loss = 5.35724092
Iteration 14, loss = 5.38684392
Iteration 15, loss = 5.65432593
Iteration 16, loss = 5.55752508
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.1, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=10,
       warm_start=False)